In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

# Set up constants
IMG_SIZE = (300, 300)  # EfficientNetB3 recommended size
BATCH_SIZE = 32
EPOCHS = 30  # Increased epochs for better learning

# Define the path to the images folder
data_dir = '/content/drive/My Drive/images'
print("Contents of data_dir:")
print(os.listdir(data_dir))



In [ ]:
# Update class names to include SCC
class_names = ['MEL', 'NV', 'BCC', 'SCC']
for class_name in class_names:
    if not os.path.isdir(os.path.join(data_dir, class_name)):
        raise ValueError(f"Folder {class_name} not found in {data_dir}")

# Set up data generators with increased augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=class_names,
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=class_names,
    shuffle=True
)

# Load pre-trained EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))



In [ ]:
# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = True

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(4, activation='softmax')(x)  # 4 classes now

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Define F1 Score metric as a class
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Compile the model with additional metrics
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall(), F1Score()])



In [ ]:
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weight_dict = dict(enumerate(class_weights))

# Train the model with class weights
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weight_dict
)

# Save the model
model.save('/content/drive/My Drive/models/skin_lesion_classifier_efficientnetb3.h5')

# Print class indices
print("Class indices:", train_generator.class_indices)


In [ ]:

# Function to predict image
def predict_image(img_path, nv_threshold=0.7):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)

    if np.argmax(prediction) == class_names.index('NV') and prediction[0][class_names.index('NV')] < nv_threshold:
        predicted_class = class_names[np.argsort(prediction[0])[-2]]
    else:
        predicted_class = class_names[np.argmax(prediction)]

    confidence = np.max(prediction)

    return predicted_class, confidence



In [ ]:
finish_dir = '/content/drive/My Drive/finish'
# Interactive prediction loop
while True:
    user_input = input("Enter an image number (1-1000) or 'q' to quit: ")

    if user_input.lower() == 'q':
        break

    try:
        image_number = int(user_input)

        for filename in os.listdir(finish_dir):
            if filename.startswith(f"{image_number}.") and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(finish_dir, filename)

                predicted_class, confidence = predict_image(img_path)

                print(f"Image: {filename}")
                print(f"Predicted class: {predicted_class}")
                print(f"Confidence: {confidence:.2f}")
                print()
                break
        else:
            print(f"No image found with number {image_number}")

    except ValueError:
        print("Invalid input. Please enter a number or 'q' to quit.")

print("Thank you for using the classifier!")